The transfer function

TODO! compare against https://camb.readthedocs.io/en/latest/camb.html#camb.get_transfer_functions

In [1]:
%matplotlib ipympl
# %matplotlib inline

In [2]:
import numpy as np
from numpy import sqrt, exp, log, sin, cos, power
import scipy.integrate as integ
from scipy.interpolate import UnivariateSpline, InterpolatedUnivariateSpline
import functools

import astropy.constants as const
import astropy.units as u
from astropy.cosmology import Planck18
from astropy.cosmology.core import Cosmology
from astropy.visualization import quantity_support

import matplotlib.pyplot as plt

from cosmic_conchometer.boltzmann import husugiyama95
from cosmic_conchometer.boltzmann.husugiyama95 import TransferFunctionHuSugiyama1995Theta0  # Theta0hatSSFixedR, Theta0hatLSFixedR
from cosmic_conchometer.utils import distances

quantity_support()

<astropy.visualization.units.quantity_support.<locals>.MplQuantityConverter at 0x7fe7246f46d0>

In [3]:
zeq = distances.z_matter_radiation_equality(Planck18)
a = (1 + zeq) / 1101  # double check!

In [4]:
from scipy.interpolate import Akima1DInterpolator, PchipInterpolator

In [5]:
%%time
Theta0 = TransferFunctionHuSugiyama1995Theta0(Planck18, a=1 / (1 + 1100),
                                              As=1e-10 * np.exp(3.04) * u.K**2,
                                              knum=int(1e5),
                                              krange=(1e-4, 0.03, 100.0) / u.Mpc,
                                              integ_kw=dict(limit=400, epsabs=2e-7, epsrel=5e-29),
                                              lgkwindow=0.1,
                                              _interp_fn=PchipInterpolator)

CPU times: user 9.05 s, sys: 190 ms, total: 9.24 s
Wall time: 9.49 s


In [6]:
# lgk, th0 = Theta0._lgk, Theta0._th0

In [7]:
fig = plt.figure(figsize=(8, 3))
ax = fig.add_subplot(111)

lgk = (Theta0._lgk.to(1/u.Mpc) + 0.001 / u.Mpc).to(u.dex(1/u.Mpc))  # offset from ticks
th0 = Theta0(lgk)

lowk = lgk < Theta0._kcross
highk = ~lowk

th0adj = np.log10(np.abs(th0)) * lgk.value ** 2

th0adj2 = np.sign(th0[highk]) * np.power(
    np.abs(th0[highk]),
    lgk[highk].value ** 2,
)

# ax.scatter(lgk.value[highk], th0adj2, s=1)
ax.plot(lgk.value[highk], th0adj2)
ax.axhline(0, c="k", ls=":")
ax.axvline(Theta0._lgk_zerox.value, c="k", ls=":")
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-7-6063645686f8>:10: RuntimeWarning: divide by zero encountered in log10
  th0adj = np.log10(np.abs(th0)) * lgk.value ** 2


In [8]:
# fig = plt.figure(figsize=(8, 3))
# ax = fig.add_subplot(111)

# th0adj2 = np.sign(th0[zerox_idx:]) * np.power(np.abs(th0), lgk.value ** 2)[zerox_idx:]

# ax.scatter(lgk[zerox_idx:], th0adj2, s=1)
# ax.plot(lgk[zerox_idx:], Theta0._fithighk(lgk[zerox_idx:]), c="k")
# plt.show();

In [9]:
Theta0.plot_theta0hat(fit=True, s=1)

# k = (10**lgk) / u.Mpc
# plt.plot(k, Theta0._signk(lgk) / 100)

plt.axhline(0, c="k")
plt.axvline(Theta0._kcross.value)
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig = plt.figure()
ax = fig.add_subplot(xscale="log", yscale="symlog",)
plt.plot(Theta0._lgk.to(1 / u.Mpc),
         Theta0(Theta0._lgk.to(1/u.Mpc) + 0.001 / u.Mpc))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …